In [27]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

In [28]:
tmp <- subset(df, select = -c(deg_cat))           #hot encoding

#define one-hot encoding function
dummy <- dummyVars(" ~ .", data=tmp)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=tmp))


final_df$deg_cat <- df$deg_cat


df <- final_df

## Data partition

In [29]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)


TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [30]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    401     402 

## Models

In [31]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [32]:

model_monmlp <- train(x=TrainData, y=TrainClasses, 
               method = "monmlp") 

** Ensemble 1 
Complex eigenvalues found for method = BFGS 
coefficients for function value 0.263627  :
  [1]  -55.28087748   89.46514288   31.11487644   57.67611836   90.87139270
  [6]  -88.29797569  -85.73079095  -22.87546875   22.89249598   50.01630531
 [11] -127.43629024  -29.50292105  -49.61386011   57.12965286  -29.01642870
 [16]   29.10725774  -44.87114445  -50.10083826   39.11660143   33.42310754
 [21]   -4.60826231    4.67701741   50.81739640  -13.15454974  -57.14714036
 [26]  -11.53986987   11.56819235   -0.05509444  -17.47868566   48.06205626
 [31]   86.18089243   17.04100009  -48.37436085    9.14349211   -9.13967073
 [36]   10.36038332  -10.37098514  -34.13124966   34.20023663   -7.66758565
 [41]  -42.28057762   30.10064397   29.11727699   49.33847552   -9.35398380
 [46]    9.35114880  113.29038379   23.41920134  132.98943045  -14.56369621
 [51]   21.82666825  -73.99500397   24.30470132 -128.02268789  -93.42101029
 [56]  -35.62611484  -10.90018249 -115.53477337  -39.9291917

In [33]:

model_mlpW <- train(x=TrainData, y=TrainClasses, 
               method = "mlpWeightDecay") 

## Results

In [34]:
print(model_monmlp)
plot(model_monmlp,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Monotone Multi-Layer Perceptron Neural Network 

803 samples
183 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 803, 803, 803, 803, 803, 803, ... 
Resampling results across tuning parameters:

  hidden1  Accuracy   Kappa    
  1        0.7073134  0.4151892
  3        0.7091515  0.4192114
  5        0.6900267  0.3809511

Tuning parameter 'n.ensemble' was held constant at a value of 1
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were hidden1 = 3 and n.ensemble = 1.


In [35]:
print(model_mlpW)
plot(model_mlpW,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Multi-Layer Perceptron 

803 samples
183 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 803, 803, 803, 803, 803, 803, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
  1     0e+00  0.7215297  0.4423820
  1     1e-04  0.7137614  0.4284871
  1     1e-01  0.4971899  0.0000000
  3     0e+00  0.7337338  0.4672028
  3     1e-04  0.7357342  0.4717679
  3     1e-01  0.5043466  0.0000000
  5     0e+00  0.7381268  0.4759565
  5     1e-04  0.7366355  0.4717909
  5     1e-01  0.5027373  0.0000000

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 5 and decay = 0.


In [36]:
# collect resamples
results <- resamples(list(monmlp=model_monmlp,  mlpWeightDecay=model_mlpW))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: monmlp, mlpWeightDecay 
Number of resamples: 25 

Accuracy 
                    Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
monmlp         0.6619718 0.6990291 0.7162630 0.7091515 0.7203947 0.7483221    0
mlpWeightDecay 0.6711864 0.7216117 0.7351916 0.7381268 0.7576792 0.7918089    0

Kappa 
                    Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
monmlp         0.3320594 0.3980898 0.4309931 0.4192114 0.4413698 0.4975947    0
mlpWeightDecay 0.3437679 0.4417420 0.4707880 0.4759565 0.5108231 0.5837584    0


## Prediction

In [37]:
pred_mlpW <- predict(model_mlpW,TestData)
pred_mlpW.prob <- predict(model_mlpW,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_mlpW) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      68      53
   NotFast   52     171
                                         
               Accuracy : 0.6948         
                 95% CI : (0.6431, 0.743)
    No Information Rate : 0.6512         
    P-Value [Acc > NIR] : 0.04937        
                                         
                  Kappa : 0.3294         
                                         
 Mcnemar's Test P-Value : 1.00000        
                                         
            Sensitivity : 0.5667         
            Specificity : 0.7634         
         Pos Pred Value : 0.5620         
         Neg Pred Value : 0.7668         
             Prevalence : 0.3488         
         Detection Rate : 0.1977         
   Detection Prevalence : 0.3517         
      Balanced Accuracy : 0.6650         
                                         
       'Positive' Class : fast           
                                   

Sensitivity          Specificity       Pos Pred Value 
           0.5666667            0.7633929            0.5619835 
      Neg Pred Value            Precision               Recall 
           0.7668161            0.5619835            0.5666667 
                  F1           Prevalence       Detection Rate 
           0.5643154            0.3488372            0.1976744 
Detection Prevalence    Balanced Accuracy 
           0.3517442            0.6650298

In [38]:
roc_knn <- multiclass.roc(TestClasses, as.numeric(pred_mlpW))
roc_knn


Setting direction: controls < cases




Call:
multiclass.roc.default(response = TestClasses, predictor = as.numeric(pred_mlpW))

Data: as.numeric(pred_mlpW) with 2 levels of TestClasses: fast, NotFast.
Multi-class area under the curve: 0.6644

In [39]:
roc_obj

ERROR: Error in eval(expr, envir, enclos): oggetto "roc_obj" non trovato


In [ ]:
pred_svmMlpW.prob